In [1]:
# TODO:
# Save the matplotlib plots
# save the models

In [ ]:
# Question :- 
# In this assignment candidate will be able to create a model that can detect coronavirus patients 
# from X-ray reports

# Just info:-
# I have already download and unzip the kaggle dataset. It's name is Covid19-dataset.

In [32]:
# Importing all the necessary libraries
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
from tensorflow.keras import layers
# from 

In [2]:
os.getcwd()

'c:\\IA_SEM_3\\TCS_Sem_3_IA1_IA2\\TCS_Sem-3\\IA_2'

In [3]:
# Saving the path of the train directory and test directory as string.

train_dir_path = "C:/IA_SEM_3/TCS_Sem_3_IA1_IA2/TCS_Sem-3/IA_2/Covid19-dataset/train"
test_dir_path = "C:/IA_SEM_3/TCS_Sem_3_IA1_IA2/TCS_Sem-3/IA_2/Covid19-dataset/test"

In [7]:
image_height = 224
image_width = 224
batch_size = 32

In [10]:
X_train = []

for folder in os.listdir(train_dir_path):
    sub_path = train_dir_path + "/" + folder
    for image in os.listdir(sub_path):
        image_train = sub_path + "/" + image
        image = cv2.imread(image_train)
        image = cv2.resize(image, (image_height, image_width)) # ResNet-50 needs an input shape of 224 x 224
        X_train.append(image)



In [11]:
X_test = []

for folder in os.listdir(test_dir_path):
    sub_test = test_dir_path + "/" + folder
    for image in os.listdir(sub_test):
        image_test = sub_test + "/" + image
        image = cv2.imread(image_test)
        image = cv2.resize(image, (image_height, image_width))
        X_test.append(image)

In [13]:
print("Maximum value of the first image in X_train: ", float(np.max(X_train[0])))
print("Minumum value of the first image in X_train: ", float(np.min(X_train[0])))

print("Maximum value of the first value in X_test: ", float(np.max(X_test[0])))
print("Minimum value of the first image in X_test: ", float(np.min(X_test[0])))

Maximum value of the first image in X_train:  255.0
Minumum value of the first image in X_train:  0.0
Maximum value of the first value in X_test:  255.0
Minimum value of the first image in X_test:  0.0


In [16]:
train_data = ImageDataGenerator(rescale = 1.0 / 255.0, validation_split = 0.2)
test_data = ImageDataGenerator(rescale = 1.0 / 255.0)

In [18]:
train_set = train_data.flow_from_directory(
    train_dir_path,
    subset = "training",
    batch_size = batch_size,
    target_size = (image_height, image_width),
    class_mode = "categorical" # sparse
)
validation_set = train_data.flow_from_directory(
    train_dir_path,
    subset = "validation",
    batch_size = batch_size,
    target_size = (image_height, image_width),
    class_mode = "categorical" # sparse
)
test_set = test_data.flow_from_directory(
    test_dir_path,
    batch_size = batch_size,
    target_size = (image_height, image_width),
    class_mode = "categorical" # sparse
)

Found 201 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [28]:
# ResNet50 Model
resnet_50 = ResNet50(
    include_top = False, 
    input_shape = (224, 224, 3),
    weights = "imagenet"
    )

for layers in resnet_50.layers:
    layers.trainable = False

In [29]:
# Early Stopping
early_stopping = EarlyStopping(
    monitor = "val_loss",
    patience = 10,
    restore_best_weights = True,
    min_delta = 0.001,
    verbose = 0
)

In [34]:
resnet_50_model = Sequential(
    [
        # layers.RandomFlip(mode = "horizontal"),
        layers.RandomFlip(mode = "horizontal"),
        layers.RandomRotation(factor = 0.2),
        resnet_50,
        layers.GlobalAveragePooling2D(),
        layers.Flatten(),
        layers.Dense(units = 512, activation = "relu"),
        layers.Dense(units = 100, activation = "relu"),
        layers.Dense(units = 3, activation = "softmax")
    ]
)

In [35]:
resnet_50_model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy", # sparse_categorical_crossentropy
    metrics = ["accuracy"]
)

In [37]:
# Fitting the ResNet-50 model

history = resnet_50_model.fit(
    train_set,
    callbacks = [early_stopping],
    steps_per_epoch = 7,
    epochs = 2, # 100
    validation_data = validation_set
)

Epoch 1/2
7/7 [==============================] - 38s 5s/step - loss: 1.1825 - accuracy: 0.4080 - val_loss: 1.0502 - val_accuracy: 0.5600
Epoch 2/2
7/7 [==============================] - 30s 4s/step - loss: 1.0186 - accuracy: 0.4776 - val_loss: 1.0011 - val_accuracy: 0.6600


In [38]:
# Evaluate the model on the test data set

loss, accuracy = resnet_50_model.evaluate(test_set, verbose = 0)

# Evaluation result
print("Test loss: ", loss)
print("Test accuracy: ", accuracy)

Test loss:  0.9607003331184387
Test accuracy:  0.6515151262283325


In [ ]:
# Plotting the Training loss and Validation Loss
 
plt.figure